In [8]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# setting up sncosmo
import sncosmo

In [10]:
import os
from copy import deepcopy

In [11]:
import gedankenLSST
import numpy as np
import pandas as pd

In [12]:
from LSSTmetrics import PerSNMetric

In [13]:
from lsst.sims.photUtils import BandpassDict

In [17]:
import analyzeSN.analyzelcFits as anf

In [18]:
# Run this once. It does not change

lsst_bp = BandpassDict.loadTotalBandpassesFromFiles()

# sncosmo Bandpasses required for fitting
throughputsdir = os.getenv('THROUGHPUTS_DIR')

from astropy.units import Unit
bandPassList = ['u', 'g', 'r', 'i', 'z', 'y']
banddir = os.path.join(os.getenv('THROUGHPUTS_DIR'), 'baseline')

for band in bandPassList:

    # setup sncosmo bandpasses
    bandfname = banddir + "/total_" + band + '.dat'


    # register the LSST bands to the SNCosmo registry
    # Not needed for LSST, but useful to compare independent codes
    # Usually the next two lines can be merged,
    # but there is an astropy bug currently which affects only OSX.
    numpyband = np.loadtxt(bandfname)
    print band
    sncosmoband = sncosmo.Bandpass(wave=numpyband[:, 0],
                                   trans=numpyband[:, 1],
                                   wave_unit=Unit('nm'),
                                   name=band)
    sncosmo.registry.register(sncosmoband, force=True)

u
g
r
i
z
y


In [19]:
lsstCadence = deepcopy(gedankenLSST.LSSTReq)

In [20]:
## THIS IS WHAT YOU NEED TO CHANGE
lsstCadence['bF'] = 1


# time window increased to 200 days to make sure LC stays within time frame
lsst_obs = gedankenLSST.GSN_Obs(mjd_center=49570., 
                                    lsstrequirements=lsstCadence,
                                    ra=58., dec=-27.,
                                    timeWindow=[-130., 150.]) 

snLSST = PerSNMetric(summarydf=lsst_obs.summary,t0=49570., raCol='ra', decCol='dec', lsst_bp=lsst_bp)

In [21]:
# function that takes a set of parameters, producing a lightcurve mathematical function
data = snLSST.SNCosmoLC() #sncosmo.Model(source='hsiao')
#model.set(z=0.5, t0=55000., amplitude=1.e-10)

20


In [12]:
## For plotting tests
# function that takes evaluates lightcurve function given a sampling strategy
#times = [54990., 55000., 55020.]
#samples = -1.*np.array(model.bandmag('desr', 'ab', times))
#plt.plot(times,samples)

In [15]:
# function that shifts lightcurve sampling, creating many samplings of same underlying lightcurve
#for x in times:
#    model.set(z=0.5, t0=x, amplitude=1.e-10)
#    sample = -1.*np.array(model.bandmag('desr', 'ab', times))
#    plt.plot(times,sample)
#plt.show()

In [24]:
# function that fits lightcurves to samples, producing some set of parameters and variances

model = sncosmo.Model(source='salt2-extended')
fit_out = sncosmo.fit_lc(data,model,['z', 't0', 'x0', 'x1', 'c'],bounds={'z':(0.3, 0.7)})

#model = sncosmo.Model(source='salt2-extended')
#mcmc_out = sncosmo.mcmc_lc(data,model,['z', 't0', 'x0', 'x1', 'c'],bounds={'z':(0.3, 0.7)})

In [38]:
params = fit_out[0].parameters
errs = fit_out[0].errors
print(params,errs)

(array([  5.04327775e-01,   4.95700436e+04,   8.90806593e-06,
         1.32465424e-02,   2.14682932e-02]), OrderedDict([('z', 0.029914892963567252), ('t0', 0.6736711627090699), ('x0', 2.8189711392232117e-07), ('x1', 0.3526768354774094), ('c', 0.05477753842357352)]))


In [40]:
def mu(params):
    x0 = params[2]
    x1 = params[3]
    c = params[4]
    return -2.5*np.log10(x0)+0.1*x1+-3.1*c

In [41]:
onemu = mu(params)
print(onemu)

12.5603143887


In [14]:
t = anf.ResChar.fromSNCosmoRes(mcmc_out)
samples = t.salt_samples()
mu = samples['mu']

In [19]:
var_mu = t.salt_samples().mu.std()*t.salt_samples().mu.std()

In [21]:
# function that produces expected value of variances for many shifted lightcurves

In [ ]:
# function that calculates bias of fit parameters relative to true parameters

In [ ]:
# use LSST cadence
# choose number of lightcurves to shift
# for each shifted lightcurve, evaluate at sampling locations (no measurement errors for now)
# for each shifted lightcurve, calculate variance of parameters, then sum them to get expected value of variance
# profit